# Data Visualization and Exploration - Python

- Jackson Cown

In [28]:
import numpy as np
import pandas as pd
import datetime as dt

### Loading the Datasets

In [4]:
# Load file paths
items_path = "../data/items.csv"
category_hierarchy_path = "../data/category_hierarchy.csv"
orders_path = "../data/orders.csv"
# Load datasets into pandas
items = pd.read_csv(items_path, sep="|")
category_hierarchy = pd.read_csv(category_hierarchy_path, sep="|")
orders = pd.read_csv(orders_path, sep="|")

In [5]:
# Check Items
items.head()

,itemID,brand,feature_1,feature_2,feature_3,feature_4,feature_5,categories
0,22665,861,4,0,490,2,66,"[2890, 855, 3908, 3909]"
1,28640,1366,10,1,537,0,101,NaN
2,13526,1090,10,0,511,0,0,"[3270, 163, 284, 1694, 12, 3837, 2422, 3595, 3..."
3,21399,1090,10,1,511,0,0,[3270]
4,8504,768,4,1,484,0,66,[2470]


In [6]:
# Check Category Hierarchy
category_hierarchy.head()

,category,parent_category
0,0,75
1,1,1499
2,2,1082
3,3,3498
4,4,1623


In [7]:
# Check Orders
orders.head()

,date,userID,itemID,order
0,2020-06-01,38769,3477,1
1,2020-06-01,42535,30474,1
2,2020-06-01,42535,15833,1
3,2020-06-01,42535,20131,1
4,2020-06-01,42535,4325,1


### Munging the Dataset

In [20]:
# Generate Dates
start = dt.datetime.strptime("2020-06-01", "%Y-%m-%d")
date_generated = pd.date_range(start, periods=245)
print(date_generated)

DatetimeIndex(['2020-06-01', '2020-06-02', '2020-06-03', '2020-06-04',
               '2020-06-05', '2020-06-06', '2020-06-07', '2020-06-08',
               '2020-06-09', '2020-06-10',
               ...
               '2021-01-22', '2021-01-23', '2021-01-24', '2021-01-25',
               '2021-01-26', '2021-01-27', '2021-01-28', '2021-01-29',
               '2021-01-30', '2021-01-31'],
              dtype='datetime64[ns]', length=245, freq='D')


In [14]:
# Define a function to map dates to number of orders
def date_order_pair():
    date_order_dict = {}
    for i, date in enumerate(date_generated):
        # Format date for comparisn
        date = date.strftime("%Y-%m-%d")
        # Compile and sum list of orders per day; update dict
        ordered = np.sum([item for item in orders.loc[orders["date"] == date].order])
        date_order_dict[i] = [date, ordered]
        print(f'Date: {date}; \nOrder Count: {ordered}\n')
    return date_order_dict

# date_order_dict = date_order_pair()

# Define a function to convert a Dict to a DataFrame and Save it as a CSV
def date_order_to_csv(date_order_dict):
    date_order_df = pd.DataFrame.from_dict(date_order_dict, orient='index', columns=["date", "num_orders"])
    date_orders_path = '../data/date_orders.csv'
    date_order_df.to_csv(date_orders_path)


#date_order_to_csv(date_order_pair())

In [15]:
# Generating DataFrame for Month-Order Pairs
date_orders_path = '../data/date_orders.csv'
date_order = pd.read_csv(date_orders_path)
date_order.head()

,Unnamed: 0,date,num_orders
0,0,2020-06-01,7148
1,1,2020-06-02,6039
2,2,2020-06-03,6781
3,3,2020-06-04,5904
4,4,2020-06-05,5118


In [16]:
months = ["06", "07", "08", "09", "10", "11", "12", "01"]
month_names = ["June", "July", "August", "September", "October", "November", "December", "January"]

# Define a function to map months to number of orders
def month_order_pair():
    # Init dict for storing month-order pairs
    month_order_dict = {}
    month_and_names = {months[i]:month_names[i] for i in range(len(months))}
    for month in months:
        for i, item in enumerate(date_order.date):
            if (item[5:7] == month):
                month_order_dict[int(month)] = month_order_dict.get(int(month), 0) + date_order.num_orders[i]
                print(f'Month: {month}; Date: {item}; Count: {date_order.num_orders[i]}')
        print(f'{month_and_names[month]} Total Orders: {month_order_dict[int(month)]}\n')
    return month_order_dict

# Define a function to convert dict to DataFrame and save as a CSV
def month_order_to_csv(month_order_dict):
    month_order_df = pd.DataFrame.from_dict(month_order_dict, orient='index', columns=["num_orders"])
    month_order_df.insert(0, "month", month_names)
    # Export month-order pairs to csv
    month_order_path = "../data/month_orders.csv"
    month_order_df.to_csv(month_order_path)

#month_order_to_csv(month_order_pair())

In [18]:
# Define a method to map the itemID to number of orders
def item_order_pair():
    item_order_dict = {}
    for i, item in enumerate(items.itemID):
        # Compile and sum list of orders per item; update dict
        ordered = np.sum([item for item in orders.loc[orders["itemID"] == item].order])
        item_order_dict[i] = [item, ordered]
        print(f'ItemID: {item}; \nOrder Count: {ordered}\n')
    return item_order_dict

# Define a method to convert a Dict to a DataFrame and Save it as a CSV
def item_order_to_csv(item_order_dict):
    item_order_df = pd.DataFrame.from_dict(item_order_dict, orient='index', columns=["itemID", "num_orders"])
    item_order_path = '../data/item_orders.csv'
    item_order_df.to_csv(item_order_path)

#item_order_to_csv(item_order_pair())

ItemID: 22665; 
Order Count: 4

ItemID: 28640; 
Order Count: 4

ItemID: 13526; 
Order Count: 279

ItemID: 21399; 
Order Count: 5

ItemID: 8504; 
Order Count: 14

ItemID: 32122; 
Order Count: 1

ItemID: 31956; 
Order Count: 1

ItemID: 6237; 
Order Count: 3

ItemID: 16971; 
Order Count: 427

ItemID: 18385; 
Order Count: 145

ItemID: 12775; 
Order Count: 83

ItemID: 8285; 
Order Count: 52

ItemID: 16238; 
Order Count: 38

ItemID: 29828; 
Order Count: 73

ItemID: 13025; 
Order Count: 12

ItemID: 18912; 
Order Count: 35

ItemID: 366; 
Order Count: 0.0

ItemID: 17432; 
Order Count: 2

ItemID: 4119; 
Order Count: 1

ItemID: 1954; 
Order Count: 3

ItemID: 14612; 
Order Count: 6

ItemID: 1373; 
Order Count: 1

ItemID: 2440; 
Order Count: 3

ItemID: 6084; 
Order Count: 1

ItemID: 31733; 
Order Count: 6

ItemID: 19652; 
Order Count: 196

ItemID: 7002; 
Order Count: 3

ItemID: 1504; 
Order Count: 31

ItemID: 3914; 
Order Count: 351

ItemID: 24190; 
Order Count: 389

ItemID: 7811; 
Order Count: 1



In [27]:
item_orders = pd.read_csv("../data/item_orders.csv")
item_orders.head()
item_orders_nlargest_path = "../data/item_orders_nlargest.csv"
item_orders.nlargest(10, "num_orders").to_csv(item_orders_nlargest_path)

In [ ]:
#